In [1]:
from idealab_tools.kinematics.kinematics import Quaternion
from typing import Dict, Union
import mujoco
from gymnasium import spaces
import gymnasium as gym
import gymnasium
import numpy as np
import numpy
from gymnasium import utils
from gymnasium.spaces import Box
import math
import os
import os
import mujoco
import numpy
import mediapy as media
import matplotlib.pyplot as plt
import math
import yaml

In [2]:
width = 800
height = 600

In [3]:
xml_template = '''
<mujoco>
 <option>
    <flag energy="enable"/>
  </option>
  <option>
     <flag gravity="enable" contact="enable" />
  </option>
  <option timestep="{ts:1.3e}"/>
  <compiler angle="degree" />
    <visual><global offwidth="{width}" offheight="{height}" /></visual>    

  <worldbody>
    <light name="top" pos="0 0 1"/>
    <camera name="world" mode="fixed" pos="0 -.5 .05" axisangle="1 0 0 90"/>
    <camera name="target" mode="targetbody" target="trunk" pos="0 -.5 .05" axisangle="1 0 0 90" />

    
    <body name="floor" pos="0 0 0">
      <geom name="floor" pos="0 0 0" size="1 1 .1" type="plane" rgba="1 0.83 0.61 0.5"/>
    </body>

    <body name="trunk" pos="0 0 1">
        <joint type="free"/>
        <geom type="box" size=".02 .02 .02" pos="0 0 0" rgba="1 0 0 1"/>
        

          <body name="leg1" pos="0 0.02 -.02">
              <joint name="j1" type="hinge" axis="0 1 0" pos="0 0 0"  damping=".01" />
              <geom type="box" size=".005 .005 .02" pos="0 0 -0.02" rgba="1 0 0 1"/>
              <body name="leg12" pos="0 0 -.04">
                  <joint name="j12" type="hinge" axis="0 1 0" pos="0 0 0" stiffness=".1" damping=".01" limited="true" range="0 180" />
                  <geom type="box" size=".005 .005 .02" pos="0 0 -0.02" rgba="1 0 0 1"/>
                  <body name="ankle1" pos="0 0 -0.04">forcelimited
                    <joint name="a1" type="hinge" axis="0 1 0"  stiffness="{k}" damping="{b}"/>
                    <geom type="box" size="{l_ankle} .02 .005" pos="0 0 0" rgba="1 0 0 1"/>
                  </body>
              </body>
          </body>
    
          <body name="leg2" pos="0 -0.02 -.02">
              <joint name="j2" type="hinge" axis="0 1 0" pos="0 0 0" damping=".01" />
              <geom type="box" size=".005 .005 .02" pos="0 0 -0.02" rgba="1 0 0 1"/>
              <body name="leg22" pos="0 0 -.04">
                  <joint name="j22" type="hinge" axis="0 1 0" pos="0 0 0" stiffness=".1" damping=".01" limited="true" range="0 180" />
                  <geom type="box" size=".005 .005 .02" pos="0 0 -0.02" rgba="1 0 0 1"/>
                  <body name="ankle2" pos="0 0 -0.04">
                    <joint name="a2" type="hinge" axis="0 1 0"  stiffness="{k}" damping="{b}"/>
                    <geom type="box" size="{l_ankle} .02 .005" pos="0 0 0" rgba="1 0 0 1"/>
                  </body>
              </body>
          </body>

    </body>
  </worldbody>

   <contact>
<!--
<exclude body1="leg1_l4" body2="leg1_l4_2" />
-->
  </contact>
  
  <actuator>
<!--
    <motor name="m1" joint="j1"/>
    <motor name="m2" joint="j2"/>
-->
    <position name="m1" joint="j1" kp="1" forcelimited="true" forcerange="-.05 .05"/>
    <position name="m2" joint="j2" kp="1" forcelimited="true" forcerange="-.05 .05"/>

  </actuator>

</mujoco>
'''

In [4]:
xml = xml_template.format(l_ankle=.03,k=.1,b=.01,ts=1e-3,width=800,height=600)

In [5]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model,width=width,height=height)
# renderer = mujoco.Renderer(model)


In [6]:
A = .25
d = 0 
ts = .25
f = .5
l1 = 0
l2 = .5

def mycontroller(model, data):
    t = data.time
    linear_act = t*d/ts
    if linear_act < d:
        linear_act = d
    pos1 = A*math.sin(2*math.pi*(f*t-l1))
    pos2 = A*math.sin(2*math.pi*(f*t-l2))
    data.ctrl = [pos1,pos2]
    return


mujoco.set_mjcb_control(mycontroller)

In [7]:
mujoco.mj_resetData(model,data)
# data.qpos[0:2] = [-45*math.pi/180,-45*math.pi/180]
# data.qvel[0:2] = [-3,0]
frames=[]

duration = 5
framerate =30

q = []
w = []
t = []
xy = []

while data.time<duration:


    mujoco.mj_step(model,data)

    q.append(data.qpos.copy())
    w.append(data.qvel.copy())
    xy.append(data.xpos.copy())
    t.append(data.time)
    
    if len(frames)<data.time*framerate:
    
        # renderer.update_scene(data)
        renderer.update_scene(data,"world")
        pixels = renderer.render()
        frames.append(pixels)

mujoco.set_mjcb_control(None)
        
media.show_video(frames,fps = framerate,width=width,height=height)
# media.write_video('pendulum.mp4',frames,fps=framerate)

This browser does not support the video tag.
